In [1]:
import pandas as pd
import numpy as np
from datetime import date
import holidays
from prophet import Prophet

import datetime
from datetime import timedelta
from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib import pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor


from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score, GridSearchCV

tscv = TimeSeriesSplit(n_splits=5)
state = np.random.RandomState(12345)

C:\Users\14488\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Вкидываем праздники, для их учёта моделькой
holidays_dict = holidays.RUS(years=(2022, 2023))
df_holidays = pd.DataFrame.from_dict(holidays_dict, orient='index') \
    .reset_index()
df_holidays = df_holidays.rename({'index':'ds', 0:'holiday'}, axis ='columns')
df_holidays['ds'] = pd.to_datetime(df_holidays.ds)
df_holidays = df_holidays.sort_values(by=['ds'])
df_holidays = df_holidays.reset_index(drop=True)

In [3]:
pr_df = pd.read_csv('D:\data_science\demand_forecast_for_products\sp_sales_task\pr_df.csv')
sales_df = pd.read_csv('D:\data_science\demand_forecast_for_products\sp_sales_task\sales_df_train.csv')
sales_submission = pd.read_csv('D:\data_science\demand_forecast_for_products\sp_sales_task\sales_submission.csv')
st_df = pd.read_csv('D:\data_science\demand_forecast_for_products\sp_sales_task\st_df.csv')

In [4]:
# Соединяем все таблицы
df_1 = pd.merge(sales_df, st_df, how='left', left_on='st_id', right_on='st_id')
df = pd.merge(df_1, pr_df, how='left', left_on='pr_sku_id', right_on='pr_sku_id')
df.dropna (how='all')


,st_id,pr_sku_id,date,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,st_is_active,pr_group_id,pr_cat_id,pr_subcat_id,pr_uom_id
0,c81e728d9d4c2f636f067f89cc14862c,c7b711619071c92bef604c7ad68380dd,2022-10-20,1,5.0,5.0,825.0,825.0,908c9a564a86426585b29f5335b619bc,429a86ff6336c144ddaf6144f52ab52c,1,1,8,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,d29b5ce9c2883f0b7e90f79071a2ca82,1
1,42a0e188f5033bc65bf8d78622277c4e,68f441429e73dbd33b81f95da31576e9,2023-01-29,0,4.0,0.0,196.0,0.0,b8b4b727d6f5d1b61fff7be687f7970f,da742b3cfca5388aaf9af28a726a3c30,1,1,12,1,c74d97b01eae257e44aa9d5bade97baf,1bc0249a6412ef49b07fe6f62e6dc8de,68f204f3838bfee4ada868b66e6a0814,17
2,1ecfb463472ec9115b10c292ef8bc986,22988026fe85fde60d8bf777ab6bc3b4,2023-03-02,0,1.0,0.0,78.0,0.0,908c9a564a86426585b29f5335b619bc,429a86ff6336c144ddaf6144f52ab52c,2,1,28,1,c74d97b01eae257e44aa9d5bade97baf,f3173935ed8ac4bf073c1bcd63171f8a,5fd72b9fc7118bd36a137d7cea8c588a,17
3,42a0e188f5033bc65bf8d78622277c4e,8e887cf8e64ab8e7173701a979476567,2023-04-05,1,0.0,0.0,69.0,69.0,b8b4b727d6f5d1b61fff7be687f7970f,da742b3cfca5388aaf9af28a726a3c30,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,a13b7a6b8e80d44d044c257a08ecccca,1
4,16a5cdae362b8d27a1d8f8c7b78b4330,e6fc24ca846339d204b0b50e9f772484,2023-02-17,0,1.0,0.0,161.0,0.0,c1f75cc0f7fe269dd0fd9bd5e24f9586,296bd0cc6e735f9d7488ebc8fbc19130,1,2,8,1,c74d97b01eae257e44aa9d5bade97baf,a368b0de8b91cfb3f91892fbf1ebd4b2,0d51481bbf77027ddb2c990cf97cd9dd,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883010,c81e728d9d4c2f636f067f89cc14862c,a0150fefa71d6606d5e642fa0e7b8ad8,2022-12-13,1,1.0,1.0,33.0,33.0,908c9a564a86426585b29f5335b619bc,429a86ff6336c144ddaf6144f52ab52c,1,1,8,1,c74d97b01eae257e44aa9d5bade97baf,c559da2ba967eb820766939a658022c8,8ec316a6bd7240d74c0567a1d9b311e5,17
883011,42a0e188f5033bc65bf8d78622277c4e,5b146d93f822459260216df28e572eca,2023-04-18,0,1.0,0.0,192.0,0.0,b8b4b727d6f5d1b61fff7be687f7970f,da742b3cfca5388aaf9af28a726a3c30,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,28fc2782ea7ef51c1104ccf7b9bea13d,6bae9a61fa499b24f28c3ddcef6ccb1b,1
883012,42a0e188f5033bc65bf8d78622277c4e,c8bd18ea08d2f47cb0a7992f11ada9de,2022-08-16,0,5.0,0.0,970.0,0.0,b8b4b727d6f5d1b61fff7be687f7970f,da742b3cfca5388aaf9af28a726a3c30,1,1,12,1,aab3238922bcc25a6f606eb525ffdc56,186a157b2992e7daed3677ce8e9fe40f,aa36dc1d45661519a5bf923429776a83,1
883013,f7e6c85504ce6e82442c770f7c8606f0,b960c6ee5b5efa12e73f810204b561a8,2023-04-23,0,4.0,0.0,262.0,0.0,3202111cf90e7c816a472aaceb72b0df,32586311f16876abf92901085bd87b99,1,1,12,1,c74d97b01eae257e44aa9d5bade97baf,c559da2ba967eb820766939a658022c8,8ec316a6bd7240d74c0567a1d9b311e5,17


In [5]:
df = df.rename(columns = {'pr_sales_in_rub' : 'y'})
df['ds'] = pd.to_datetime(df['date'])

In [6]:
df['fe_st_id'] = df['st_id'].map(df['st_id'].value_counts(normalize=False))
df['fe_pr_sku_id'] = df['pr_sku_id'].map(df['pr_sku_id'].value_counts(normalize=False))
df['fe_st_city_id'] = df['st_city_id'].map(df['st_city_id'].value_counts(normalize=False))
df['fe_st_division_code'] = df['st_division_code'].map(df['st_division_code'].value_counts(normalize=False))
df['fe_pr_group_id'] = df['pr_group_id'].map(df['pr_group_id'].value_counts(normalize=False))
df['fe_pr_cat_id'] = df['pr_cat_id'].map(df['pr_cat_id'].value_counts(normalize=False))
df['fe_pr_subcat_id'] = df['pr_subcat_id'].map(df['pr_subcat_id'].value_counts(normalize=False))

In [7]:
df = df.drop(['st_id', 'pr_sku_id', 'st_city_id',
              'st_division_code', 'pr_group_id',
                  'pr_cat_id', 'pr_subcat_id',
                  'pr_promo_sales_in_units',
                   'pr_promo_sales_in_rub', 'date'], axis=1)

In [8]:
date_lag = 15
predictions_period = df['ds'].max() - timedelta(date_lag)
train = df.loc[df['ds'] < predictions_period]
test = df.loc[df['ds'] >= predictions_period]

In [9]:
print(train['ds'].max())
print(train['ds'].min())
print(test['ds'].max())
print(test['ds'].min())

2023-07-02 00:00:00
2022-08-01 00:00:00
2023-07-18 00:00:00
2023-07-03 00:00:00


In [10]:
X_train = train.drop(['y', 'ds'], axis=1)
y_train = train['y']

In [11]:
# model_forest = RandomForestRegressor().fit(X_train, y_train)

In [12]:
model_lgb = LGBMRegressor().fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017794 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 750
[LightGBM] [Info] Number of data points in the train set: 841234, number of used features: 14
[LightGBM] [Info] Start training from score 593.528538


In [13]:
# model_cbr = CatBoostRegressor(loss_function='RMSE'
# ).fit(X_train, y_train, verbose=False)

In [24]:
model_Prophet = Prophet(holidays=df_holidays, daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True).fit(train)

20:42:26 - cmdstanpy - INFO - Chain [1] start processing
20:42:57 - cmdstanpy - INFO - Chain [1] done processing


In [15]:
sales_submission_test = pd.DataFrame(columns=['st_id', 'pr_sku_id', 'date'])
df_1 = pd.DataFrame(columns=['st_id', 'pr_sku_id', 'date', 'target'])

In [16]:
# X_test = test.drop(['y', 'ds'], axis=1)

In [17]:
# df_1['target_model_lgb'] = model_lgb.predict(X_test)

In [18]:
# df_1['target_model_cbr'] = model_cbr.predict(X_test)

In [19]:
# df_1['target_model_gbr'] = model_gbr.predict(X_test)

In [20]:
# future = model_Prophet.make_future_dataframe(periods=date_lag)
# forecast = model_Prophet.predict(future)
# df_1['target_model_Prophet'] = forecast['yhat']
# forecast

In [21]:
a = set(test['fe_st_id'])
b = set(test['fe_pr_sku_id'])

In [22]:
df = df.dropna()

In [23]:
for i in a:
    for j in b:
        test_pred = test[(test['fe_st_id'] == i) & (test['fe_pr_sku_id'] == j)]
       
        df_1 = pd.DataFrame(columns=['st_id', 'pr_sku_id', 'date', 'y'])

        date_lag = 15
        predictions_period = test_pred['ds'].max() - timedelta(date_lag)
        train_1 = test_pred.loc[test_pred['ds'] < predictions_period]
        test_1 = test_pred.loc[test_pred['ds'] >= predictions_period]
        # print(test_1)

        df_1['st_id'], df_1['pr_sku_id'], df_1['date'], df_1['y'] = test_pred['fe_st_id'], test_pred['fe_pr_sku_id'], test_pred['ds'], test_pred['y']
        # # X_train = train.drop(['y', 'ds'], axis=1)
        # # y_train = train['y']
        # X_test = test.drop(['y', 'ds'], axis=1)

        # model_lgb = LGBMRegressor().fit(X_train, y_train)
        # df_1['target_model_lgb'] = model_lgb.predict(X_test)

        # model_cbr = CatBoostRegressor(loss_function='RMSE').fit(X_train, y_train, verbose=False)
        # df_1['target_model_cbr'] = model_cbr.predict(X_test)

        # model_Prophet = Prophet(holidays=df_holidays, daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True).fit(train)
        future = model_Prophet.make_future_dataframe(periods=date_lag)
        forecast = model_Prophet.predict(future)
        df_1['target_model_Prophet'] = forecast['yhat']
        
        sales_submission_test = pd.concat([sales_submission_test, df_1], ignore_index=False)

ValueError: Length of values (41781) does not match length of index (22)

In [ ]:
sales_submission_test.groupby(sales_submission_test['date']).count()